In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from tqdm.notebook import tqdm

from simulator import Simulator
from strategy import Strategy
from typing import Optional

In [2]:
class GridSearchStrategy(Strategy):
    required_rows = 1
    def __init__(self, rows, const):
        self.required_rows = rows
        self.const = const
        
        
    def compute_target_position(self, current_data: pd.DataFrame, current_position: float) -> Optional[float]:
        avg_price = current_data['price'].mean()
        current_price = current_data['price'][-1]

        target_position = current_position + (avg_price - current_price) / self.const

        return target_position

In [9]:
sim_results2 = pd.read_csv('sim_result.csv')

In [10]:
sim_results2

,timestamp,cash,position,fee,profit,cumulative_fees,profit_without_fees
0,"Index(['price', 'volume'], dtype='object')",-2.409403e+04,1.435572,9.633760,-9.633760,9.633760,-1.156408e-12
1,"Index(['price', 'volume'], dtype='object')",-4.029120e+04,2.400598,6.476275,-15.277403,16.110035,8.326316e-01
2,"Index(['price', 'volume'], dtype='object')",-6.047529e+04,3.600738,8.070409,58.152503,24.180443,8.233295e+01
3,"Index(['price', 'volume'], dtype='object')",-6.596469e+04,3.927293,2.194881,26.647617,26.375324,5.302294e+01
4,"Index(['price', 'volume'], dtype='object')",-7.172118e+04,4.269874,2.301679,-1.613472,28.677003,2.706353e+01
...,...,...,...,...,...,...,...
125246,"Index(['price', 'volume'], dtype='object')",-3.485164e+06,196.757412,0.668347,499791.850713,132539.710832,6.323316e+05
125247,"Index(['price', 'volume'], dtype='object')",-3.481822e+06,196.592278,1.337096,497716.690493,132541.047928,6.302577e+05
125248,"Index(['price', 'volume'], dtype='object')",-3.477450e+06,196.376109,1.749507,495851.246189,132542.797435,6.283940e+05
125249,"Index(['price', 'volume'], dtype='object')",-3.473502e+06,196.180759,1.579737,492636.953302,132544.377173,6.251813e+05


In [11]:
df

,cash,position,fee,profit,cumulative_fees,profit_without_fees
timestamp,,,,,,
"[price, volume]",-1.042864e+04,0.621360,4.169789,-4.169789,4.169789,4.414247e-13
"[price, volume]",-2.132331e+04,1.270464,4.356125,-8.165525,8.525914,3.603887e-01
"[price, volume]",-3.222645e+04,1.918761,4.359513,30.607213,12.885427,4.349264e+01
"[price, volume]",-3.771585e+04,2.245317,2.194881,12.793617,15.080308,2.787392e+01
"[price, volume]",-4.816863e+04,2.867381,4.179440,-6.227367,19.259747,1.303238e+01
...,...,...,...,...,...,...
"[price, volume]",2.335252e+06,-97.546187,1.117714,359635.861961,79670.816775,4.393067e+05
"[price, volume]",2.332164e+06,-97.393669,1.234944,360662.763831,79672.051719,4.403348e+05
"[price, volume]",2.328865e+06,-97.230702,1.318933,361584.736885,79673.370651,4.412581e+05


In [5]:
strategy = GridSearchStrategy(24*60*2, 1000)

t = Simulator(strategy)

start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2022, 3, 30)
t.run(start_date, end_date)

df = t.get_result_dataframe()

In [6]:
df['profit'][-1]

362225.2217755569

In [15]:
rows_space = [(24 * 60 * 24) + 60 * i for i in range(21)]
const_space = [100 * i for i in range(1, 26)]

In [16]:
36000 / 60 / 24

25.0

In [17]:
stats = {}
best_row = None
best_const = None
best_profit = -1e10

for row in tqdm(rows_space):
    stats[row] = {}
    
    for const in tqdm(const_space, leave=False):
        
        strategy = GridSearchStrategy(row, const)

        t = Simulator(strategy)

        start_date = datetime.datetime(2022, 1, 1)
        end_date = datetime.datetime(2022, 3, 30)
        t.run(start_date, end_date)

        df = t.get_result_dataframe()
        
        stats[row][const] = df['profit'][-1]
        if df['profit'][-1] > best_profit:
            best_profit = df['profit'][-1]
            best_row = row
            best_const = const
            print('profit:', best_profit)
            print(f'\trow: {best_row} const: {best_const}')

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

profit: 2370907.6501973555
	row: 34560 const: 100
profit: 2523845.205722433
	row: 34560 const: 200
profit: 2610710.058833402
	row: 34560 const: 300
profit: 2660119.378598712
	row: 34560 const: 400
profit: 2692797.8963638307
	row: 34560 const: 500
profit: 2740905.2890157844
	row: 34560 const: 600
profit: 2763162.940780533
	row: 34560 const: 700
profit: 2773252.0038414323
	row: 34560 const: 800
profit: 2801040.9281612434
	row: 34560 const: 900
profit: 2821780.376883417
	row: 34560 const: 1000
profit: 2836105.9452491305
	row: 34560 const: 1100
profit: 2853028.0505698896
	row: 34560 const: 1200


  0%|          | 0/25 [00:00<?, ?it/s]

profit: 2854588.32732709
	row: 34620 const: 1200


...

profit: 2855827.160710689
	row: 34680 const: 1200


  0%|          | 0/25 [00:00<?, ?it/s]

profit: 2856982.574674458
	row: 34740 const: 1200


  0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
pd.DataFrame.from_dict(stats)

,720,1440,2160,2880,3600,4320,5040,5760,6480,7200,7920,8640,9360,10080,10800,11520
500,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
1000,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
1500,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
2000,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
2500,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
3000,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
3500,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
4000,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
4500,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
5000,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776,362225.221776
